In [8]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, Iterable, Optional, Tuple, Union, List

import numpy as np
import pandas as pd
import yfinance as yf

# Optional: pretty display in notebooks
try:
    from IPython.display import display
except Exception:  # pragma: no cover
    display = None


def _as_datetime(x):
    if x is None:
        return None
    return pd.to_datetime(x)


def _fetch_prices(
    tickers,
    start: str,
    end: str | None,
    auto_adjust: bool = True,
) -> pd.DataFrame:
    """
    Robust price fetcher for yfinance across:
      - single vs multiple tickers
      - MultiIndex ticker/field ordering
      - auto_adjust True/False

    Returns a DataFrame with columns = tickers, values = adjusted close series.
    """
    tickers = list(tickers)

    df = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        auto_adjust=auto_adjust,
        progress=False,
        group_by="ticker",
        threads=True,
    )

    if df is None or df.empty:
        return pd.DataFrame()

    # Decide which field we want
    # auto_adjust=True => "Close" is already adjusted
    wanted = "Close" if auto_adjust else "Adj Close"

    # --------
    # MultiIndex case
    # --------
    if isinstance(df.columns, pd.MultiIndex):
        lvl0 = df.columns.get_level_values(0)
        lvl1 = df.columns.get_level_values(1)

        # Case A: (ticker, field)
        if wanted in set(lvl1):
            px = df.xs(wanted, level=1, axis=1)
        # Case B: (field, ticker)
        elif wanted in set(lvl0):
            px = df.xs(wanted, level=0, axis=1)
        else:
            # Fallback: try Close then Adj Close
            for alt in ["Close", "Adj Close"]:
                if alt in set(lvl1):
                    px = df.xs(alt, level=1, axis=1)
                    break
                if alt in set(lvl0):
                    px = df.xs(alt, level=0, axis=1)
                    break
            else:
                raise KeyError(f"Could not find Close/Adj Close in columns: {df.columns}")

        px = px.copy()
        px.columns = [c.upper() for c in px.columns]  # normalize
        return px.dropna(how="all")

    # --------
    # Single-ticker (flat columns) case
    # --------
    if wanted in df.columns:
        s = df[wanted].copy()
    elif (not auto_adjust) and ("Close" in df.columns):
        # fallback if Adj Close missing
        s = df["Close"].copy()
    else:
        raise KeyError(f"Could not find '{wanted}' (or fallback) in columns: {df.columns.tolist()}")

    # If single ticker, return as DataFrame with that ticker name
    name = tickers[0].upper() if len(tickers) == 1 else "PRICE"
    return pd.DataFrame({name: s}).dropna(how="all")


def _returns(prices: pd.Series) -> pd.Series:
    r = prices.pct_change().replace([np.inf, -np.inf], np.nan)
    return r.dropna()


def _max_drawdown(prices: pd.Series) -> float:
    peak = prices.cummax()
    dd = prices / peak - 1.0
    return float(dd.min())


def _drawdown_series(prices: pd.Series) -> pd.Series:
    peak = prices.cummax()
    dd = prices / peak - 1.0
    return dd


def _ulcer_index(prices: pd.Series) -> float:
    dd = _drawdown_series(prices)
    # ulcer index uses drawdown in percent; square, average, sqrt
    dd_pct = (dd * 100.0).clip(upper=0)
    ui = float(np.sqrt(np.mean(np.square(dd_pct.values))))
    return ui


def _cagr(prices: pd.Series) -> float:
    if prices.shape[0] < 2:
        return np.nan
    start_val = float(prices.iloc[0])
    end_val = float(prices.iloc[-1])
    if start_val <= 0 or end_val <= 0:
        return np.nan
    days = (prices.index[-1] - prices.index[0]).days
    if days <= 0:
        return np.nan
    years = days / 365.25
    return float((end_val / start_val) ** (1 / years) - 1)


def _ann_vol(returns: pd.Series, periods_per_year: int = 252) -> float:
    if len(returns) < 2:
        return np.nan
    return float(returns.std(ddof=1) * np.sqrt(periods_per_year))


def _martin_ratio(prices: pd.Series, rf: float = 0.0) -> float:
    # Martin = (annualized return - rf)) / Ulcer Index
    ui = _ulcer_index(prices)
    if ui == 0 or np.isnan(ui):
        return np.nan
    ann = _cagr(prices)
    return float((ann - rf) / (ui / 100.0))  # ui was percent-based


def _time_underwater_stats(prices: pd.Series) -> Tuple[float, float, float, float]:
    """
    Returns:
      pct_days_underwater, avg_underwater_days, p95_underwater_days, max_underwater_days
    Underwater = price < prior peak.
    """
    if prices.shape[0] < 2:
        return (np.nan, np.nan, np.nan, np.nan)

    peak = prices.cummax()
    underwater = prices < peak

    # Identify contiguous underwater runs
    runs = []
    in_run = False
    run_start = None

    for dt, uw in underwater.items():
        if uw and not in_run:
            in_run = True
            run_start = dt
        elif (not uw) and in_run:
            # run ends at previous date; count business days in slice
            in_run = False
            run_end = dt
            # count rows between run_start and the day before run_end
            run_len = prices.loc[run_start:run_end].shape[0] - 1
            runs.append(max(run_len, 1))
            run_start = None

    if in_run and run_start is not None:
        run_len = prices.loc[run_start:].shape[0]
        runs.append(max(run_len, 1))

    pct = float(underwater.mean())
    if len(runs) == 0:
        return (pct, 0.0, 0.0, 0.0)

    runs_arr = np.array(runs, dtype=float)
    return (
        pct,
        float(np.mean(runs_arr)),
        float(np.percentile(runs_arr, 95)),
        float(np.max(runs_arr)),
    )


def _recovery_metrics(prices: pd.Series) -> Tuple[float, float, float, float]:
    """
    Period-local recovery notion:
      - Find the maximum drawdown peak->trough in the window.
      - Define "recovery" as trough -> reclaim that peak.
      Outputs:
        ttr_days, recovery_slope_annualized, recovery_efficiency, did_recover(0/1)
    """
    if prices.shape[0] < 5:
        return (np.nan, np.nan, np.nan, 0.0)

    peak = prices.cummax()
    dd = prices / peak - 1.0

    # Locate trough of worst drawdown
    trough_dt = dd.idxmin()
    trough_price = float(prices.loc[trough_dt])

    # Peak before trough that defines this drawdown
    pre = prices.loc[:trough_dt]
    if pre.shape[0] < 2:
        return (np.nan, np.nan, np.nan, 0.0)

    peak_dt = pre.idxmax()
    peak_price = float(prices.loc[peak_dt])

    if peak_price <= 0 or trough_price <= 0:
        return (np.nan, np.nan, np.nan, 0.0)

    max_dd = float((trough_price / peak_price) - 1.0)  # negative

    # Find first date after trough that reaches prior peak
    post = prices.loc[trough_dt:]
    recovered = post[post >= peak_price]
    if recovered.empty:
        # No recovery to old peak within window
        # Still can compute "slope" to end of window as partial recovery
        end_price = float(prices.iloc[-1])
        days = max((prices.index[-1] - trough_dt).days, 1)
        daily_slope = float(np.log(end_price / trough_price) / days) if end_price > 0 else np.nan
        ann_slope = float(np.exp(daily_slope * 252) - 1) if not np.isnan(daily_slope) else np.nan
        # Efficiency: partial return / abs(dd)
        partial_return = float((end_price / trough_price) - 1.0)
        eff = float(partial_return / abs(max_dd)) if max_dd != 0 else np.nan
        return (np.nan, ann_slope, eff, 0.0)

    rec_dt = recovered.index[0]
    ttr_days = max((rec_dt - trough_dt).days, 1)

    # log slope from trough to recovered peak
    daily_slope = float(np.log(peak_price / trough_price) / ttr_days)
    ann_slope = float(np.exp(daily_slope * 252) - 1)
    # recovery return from trough to peak
    rec_return = float((peak_price / trough_price) - 1.0)
    eff = float(rec_return / abs(max_dd)) if max_dd != 0 else np.nan
    return (float(ttr_days), ann_slope, eff, 1.0)


def _capital_efficiency_index(cagr: float, max_dd: float) -> float:
    """
    Capital Efficiency: CAGR / |MaxDD|
    Higher = better returns per unit of drawdown risk
    """
    if np.isnan(cagr) or np.isnan(max_dd) or max_dd >= 0:
        return np.nan
    return float(cagr / abs(max_dd))


def _volatility_drag_ratio(prices: pd.Series, returns: pd.Series) -> float:
    """
    Volatility Drag: geometric mean / arithmetic mean
    Ratio of actual compounded return to simple average return.
    Lower drag (closer to 1.0) is better.
    """
    if len(returns) < 2:
        return np.nan
    
    # Arithmetic mean return
    arith_mean = float(returns.mean())
    
    # Geometric mean: (ending / starting) ^ (1/n) - 1
    if prices.iloc[0] <= 0 or prices.iloc[-1] <= 0:
        return np.nan
    
    total_return = float(prices.iloc[-1] / prices.iloc[0]) - 1.0
    n_periods = len(returns)
    geom_mean = float((1 + total_return) ** (1 / n_periods) - 1)
    
    if arith_mean == 0:
        return np.nan
    return float(geom_mean / arith_mean)


def _pct_time_near_highs(prices: pd.Series, threshold_pct: float = 0.05) -> float:
    """
    % Time Near Highs: Percentage of days within threshold_pct of rolling peak
    threshold_pct: e.g., 0.05 for within 5% of peak
    """
    if prices.shape[0] < 2:
        return np.nan
    
    peak = prices.cummax()
    pct_below = (prices / peak - 1.0)  # negative values = below peak
    near_high = pct_below >= -threshold_pct
    
    return float(near_high.mean())


def _trend_persistence(returns: pd.Series) -> float:
    """
    Trend Persistence: % of days where return has same sign as previous day
    Higher = stronger momentum/trend, Lower = more mean reversion
    """
    if len(returns) < 2:
        return np.nan
    
    sign_change = np.sign(returns.values[:-1]) == np.sign(returns.values[1:])
    # Only count where neither is zero
    valid = (returns.values[:-1] != 0) & (returns.values[1:] != 0)
    
    if valid.sum() == 0:
        return np.nan
    
    return float(sign_change[valid].mean())


def _expected_shortfall(returns: pd.Series, percentile: float = 5.0) -> float:
    """
    Expected Shortfall (CVaR): Average of worst X% of returns
    percentile: e.g., 5.0 for worst 5%
    """
    if len(returns) < 10:
        return np.nan
    
    threshold = np.percentile(returns.values, percentile)
    worst = returns[returns <= threshold]
    
    if len(worst) == 0:
        return np.nan
    
    return float(worst.mean())


def _capture_ratio(etf_prices: pd.Series, bench_prices: pd.Series, side: str) -> float:
    """
    Upside/Downside capture within window:
      - Uses daily returns
      - For upside: only days benchmark return > 0
      - For downside: only days benchmark return < 0
      Ratio of compounded returns over those selected days
    """
    etf_r = _returns(etf_prices)
    bmk_r = _returns(bench_prices)

    aligned = pd.concat([etf_r.rename("etf"), bmk_r.rename("bmk")], axis=1).dropna()
    
    if aligned.empty:
        return np.nan

    if side == "up":
        sel = aligned[aligned["bmk"] > 0]
    elif side == "down":
        sel = aligned[aligned["bmk"] < 0]
    else:
        raise ValueError("side must be 'up' or 'down'")

    if sel.empty:
        return np.nan

    etf_comp = float(np.prod(1.0 + sel["etf"].values) - 1.0)
    bmk_comp = float(np.prod(1.0 + sel["bmk"].values) - 1.0)
    if bmk_comp == 0:
        return np.nan
    return float(etf_comp / bmk_comp)


# --- Helper to normalize period values ---
def _normalize_period(val: Union[Tuple[str, Optional[str]], List[Optional[str]]]) -> Tuple[str, Optional[str]]:
    """
    Accepts period value as:
      - [start, end]
      - [start, None]
      - [start, end, trough]  (third element ignored)
    Returns (start, end) with end possibly None.
    """
    if isinstance(val, (list, tuple)):
        if len(val) >= 2:
            return val[0], val[1]
        raise ValueError("Period value must have at least [start, end_or_None]")
    raise ValueError(f"Unsupported period value type: {type(val)}")


def _period_metrics_for_series(
    etf_prices: pd.Series,
    bench_prices: Optional[pd.Series] = None,
    rf: float = 0.0,
) -> Dict[str, float]:
    r = _returns(etf_prices)

    maxdd = _max_drawdown(etf_prices)
    ui = _ulcer_index(etf_prices)
    cagr = _cagr(etf_prices)
    vol = _ann_vol(r)
    martin = _martin_ratio(etf_prices, rf=rf)

    pct_uw, avg_uw, p95_uw, max_uw = _time_underwater_stats(etf_prices)
    ttr, rec_slope, rec_eff, did_rec = _recovery_metrics(etf_prices)

    # New metrics
    cap_eff = _capital_efficiency_index(cagr, maxdd)
    vol_drag = _volatility_drag_ratio(etf_prices, r)
    pct_near_highs = _pct_time_near_highs(etf_prices)
    trend_pers = _trend_persistence(r)
    cvar = _expected_shortfall(r)

    # Period-level relative metrics (not rolling - calculated over full period)
    periods_per_year = 252
    period_corr = np.nan
    period_beta = np.nan
    period_alpha_per = np.nan
    period_r2 = np.nan
    te_ann = np.nan
    ir = np.nan
    cond_corr_up = np.nan
    cond_corr_down = np.nan
    period_alpha_ann = np.nan

    if bench_prices is not None:
        bmk_r = _returns(bench_prices)
        # Align returns
        aligned = pd.concat([r.rename("etf"), bmk_r.rename("bmk")], axis=1).dropna()
        if len(aligned) >= 10:  # Need minimum data points
            etf_r = aligned["etf"]
            bmk_r = aligned["bmk"]
            
            # Full period correlation
            period_corr = float(etf_r.corr(bmk_r))
            
            # Full period beta/alpha/R2 using simple linear regression
            # beta = cov(etf, bmk) / var(bmk)
            cov_xy = etf_r.cov(bmk_r)
            var_x = bmk_r.var()
            
            if var_x > 0:
                period_beta = float(cov_xy / var_x)
                # alpha = mean(etf) - beta * mean(bmk)
                period_alpha_per = float(etf_r.mean() - period_beta * bmk_r.mean())
                # R² = correlation²
                period_r2 = float(period_corr ** 2)
                
                # Annualized alpha (simple scaling - for daily returns)
                period_alpha_ann = float(period_alpha_per * periods_per_year)
                
                # Tracking error: std(etf - bmk) annualized
                active_returns = etf_r - bmk_r
                te_ann = float(active_returns.std(ddof=1) * np.sqrt(periods_per_year))
                
                # Information ratio: annualized_alpha / tracking_error
                if te_ann > 0:
                    ir = float(period_alpha_ann / te_ann)
            
            # Conditional correlations (full period)
            # Correlation when benchmark is up
            up_mask = bmk_r > 0
            if up_mask.sum() >= 10:
                cond_corr_up = float(etf_r[up_mask].corr(bmk_r[up_mask]))
            
            # Correlation when benchmark is down
            down_mask = bmk_r < 0
            if down_mask.sum() >= 10:
                cond_corr_down = float(etf_r[down_mask].corr(bmk_r[down_mask]))

    out = {
        "CAGR": cagr,
        "AnnVol": vol,
        "MaxDD": maxdd,
        "UlcerIndex": ui,
        "Martin": martin,
        "CapitalEfficiency": cap_eff,
        "VolatilityDrag": vol_drag,
        "PctTimeNearHighs": pct_near_highs,
        "TrendPersistence": trend_pers,
        "ExpectedShortfall": cvar,
        "PctDaysUnderwater": pct_uw,
        "AvgUnderwaterDays": avg_uw,
        "P95UnderwaterDays": p95_uw,
        "MaxUnderwaterDays": max_uw,
        "TTR_Days": ttr,
        "RecoverySlope_Ann": rec_slope,
        "RecoveryEfficiency": rec_eff,
        "RecoveredToPeak_inWindow": did_rec,
        # Period-level relative metrics (vs benchmark)
        "PeriodCorr": period_corr,
        "PeriodBeta": period_beta,
        "PeriodAlpha_per_period": period_alpha_per,
        "PeriodAlpha_Ann": period_alpha_ann,
        "PeriodR2": period_r2,
        "TrackingError_Ann": te_ann,
        "InformationRatio": ir,
        "ConditionalCorr_Up": cond_corr_up,
        "ConditionalCorr_Down": cond_corr_down,
    }

    if bench_prices is not None:
        out["UpsideCapture"] = _capture_ratio(etf_prices, bench_prices, "up")
        out["DownsideCapture"] = _capture_ratio(etf_prices, bench_prices, "down")

    return out


def _format_styler(df: pd.DataFrame) -> "pd.io.formats.style.Styler":
    pct_cols = [c for c in df.columns if c in {
        "CAGR", "AnnVol", "MaxDD", "PctTimeNearHighs", "TrendPersistence",
        "RecoverySlope_Ann", "PeriodAlpha_per_period", "PeriodAlpha_Ann", "TrackingError_Ann"
    }]
    ratio_cols = [c for c in df.columns if "Capture" in c or c in {
        "Martin", "RecoveryEfficiency", "CapitalEfficiency", "VolatilityDrag",
        "InformationRatio", "PeriodR2"
    }]
    corr_beta_cols = [c for c in df.columns if c in {
        "PeriodCorr", "ConditionalCorr_Up", "ConditionalCorr_Down", "PeriodBeta"
    }]
    num_cols = [c for c in df.columns if c not in pct_cols + ratio_cols + corr_beta_cols]

    sty = df.style

    # Formats
    fmt = {c: "{:.2%}" for c in pct_cols}
    fmt.update({c: "{:.3f}" for c in ratio_cols})
    fmt.update({c: "{:.3f}" for c in corr_beta_cols})
    fmt.update({c: "{:.2f}" for c in num_cols if c not in fmt})

    # Special-case some columns
    if "UlcerIndex" in df.columns:
        fmt["UlcerIndex"] = "{:.2f}"
    if "PctDaysUnderwater" in df.columns:
        fmt["PctDaysUnderwater"] = "{:.1%}"
    if "ExpectedShortfall" in df.columns:
        fmt["ExpectedShortfall"] = "{:.2%}"
    for c in ["AvgUnderwaterDays", "P95UnderwaterDays", "MaxUnderwaterDays", "TTR_Days"]:
        if c in df.columns:
            fmt[c] = "{:.0f}"
    if "RecoveredToPeak_inWindow" in df.columns:
        fmt["RecoveredToPeak_inWindow"] = "{:.0f}"
    
    # Add DataAvailability column formatting
    if "DataAvailability" in df.columns:
        fmt["DataAvailability"] = "{:.1%}"

    sty = sty.format(fmt, na_rep="—")

    # Only apply gradient to columns that have at least one non-NaN value
    gradient_cols = []
    for c in df.columns:
        if c != "DataAvailability" and df[c].notna().any():
            gradient_cols.append(c)
    
    if gradient_cols:
        sty = sty.background_gradient(subset=gradient_cols)

    if "MaxDD" in df.columns:
        sty = sty.map(lambda v: "font-weight:600" if pd.notna(v) and v < 0 else "", subset=["MaxDD"])

    return sty


def analyze_regimes(
    tickers: Union[str, Iterable[str]],
    periods: Dict[str, Union[Tuple[str, Optional[str]], List[Optional[str]]]],
    benchmark: Optional[str] = "VTI",
    rf: float = 0.0,
    auto_adjust: bool = True,
    display_results: bool = True,
) -> Dict[str, pd.DataFrame]:
    """
    Analyze ETF behavior over multiple named time windows.

    periods: dict mapping name -> [start, end] or [start, end, trough].
    The third element (trough) is ignored; only start/end are used.

    **KEY FEATURE**: Handles ETFs with different inception dates by:
    - Using whatever data is available within each period
    - Adding a "DataAvailability" column showing % of period covered
    - Computing metrics based on the actual overlapping dates

    Usage:
      analyze_regimes(["QQQ","SPY","USMV"], periods=periods_dict, benchmark="VTI")

    Args:
      tickers: one ticker or list of tickers to analyze
      periods: dict mapping name -> (start, end). end can be None to mean "today"
      benchmark: benchmark ticker for upside/downside capture (None disables)
      rf: risk-free rate used in Martin ratio (annual, e.g. 0.02)
      auto_adjust: pass through to yfinance download
      display_results: if True and in IPython, displays styled tables

    Returns:
      dict mapping period name -> DataFrame (rows=tickers, cols=metrics + DataAvailability)
    """
    if isinstance(tickers, str):
        tick_list = [tickers]
    else:
        tick_list = list(tickers)

    results: Dict[str, pd.DataFrame] = {}

    for pname, pval in periods.items():
        # Normalize period value to (start, end)
        pstart, pend = _normalize_period(pval)

        pstart_dt = _as_datetime(pstart)
        pend_dt = _as_datetime(pend)

        if pstart_dt is None:
            raise ValueError(f"Period '{pname}' missing start date.")
        if pend_dt is not None and pend_dt <= pstart_dt:
            raise ValueError(f"Period '{pname}' has end <= start.")

        # Fetch prices for ETFs + benchmark in one call when possible
        fetch_list = tick_list.copy()
        bmk = None
        if benchmark:
            fetch_list = sorted(set(fetch_list + [benchmark]))
            bmk = benchmark

        # Fetch with wider date range to capture all possible data
        px = _fetch_prices(
            fetch_list, 
            start=str(pstart_dt.date()), 
            end=(None if pend_dt is None else str(pend_dt.date())), 
            auto_adjust=auto_adjust
        )

        if px.empty:
            continue

        # Calculate full period trading days for availability calculation
        full_period_end = pend_dt if pend_dt is not None else px.index.max()
        full_period_mask = (px.index >= pstart_dt) & (px.index <= full_period_end)
        full_period_days = full_period_mask.sum()

        rows = []
        idx = []

        bench_series = px[bmk].dropna() if (bmk and bmk in px.columns) else None

        for t in tick_list:
            if t not in px.columns:
                continue
            
            # Get ticker series and filter to period
            s_full = px[t].dropna()
            
            # Find actual overlap with requested period
            s = s_full.loc[pstart_dt: full_period_end]
            
            if len(s) < 10:  # Need minimum data for meaningful analysis
                continue
            
            # Calculate data availability
            actual_days = len(s)
            data_availability = actual_days / full_period_days if full_period_days > 0 else 0.0
            
            # Get benchmark data aligned to ETF dates
            b = None
            if bench_series is not None:
                b = bench_series.reindex(s.index).dropna()
                # Re-align s to b to ensure matching dates
                if len(b) > 0:
                    common_idx = s.index.intersection(b.index)
                    s = s.loc[common_idx]
                    b = b.loc[common_idx]

            if len(s) < 10:
                continue

            metrics = _period_metrics_for_series(
                s, 
                bench_prices=b if b is not None else None, 
                rf=rf
            )
            
            # Add data availability to metrics
            metrics["DataAvailability"] = data_availability
            
            rows.append(metrics)
            idx.append(t)

        if not rows:
            continue

        df = pd.DataFrame(rows, index=idx)

        # Sort columns with DataAvailability first
        col_order = [
            "DataAvailability",  # Show this first so users know coverage
            "CAGR", "AnnVol", "Martin", "CapitalEfficiency",
            "MaxDD", "UlcerIndex",
            "VolatilityDrag", "PctTimeNearHighs", "TrendPersistence",
            "PctDaysUnderwater", "AvgUnderwaterDays", "P95UnderwaterDays", "MaxUnderwaterDays",
            "TTR_Days", "RecoverySlope_Ann", "RecoveryEfficiency", "RecoveredToPeak_inWindow",
            "ExpectedShortfall",
            # Rolling analytics
            "RollingCorr", "ConditionalCorr_Up", "ConditionalCorr_Down",
            "RollingBeta", "RollingR2",
            "RollingAlpha_per_period", "RollingAlpha_Ann",
            "TrackingError_Ann", "InformationRatio",
            # Capture
            "UpsideCapture", "DownsideCapture",
        ]
        df = df[[c for c in col_order if c in df.columns] + [c for c in df.columns if c not in col_order]]

        results[pname] = df

        if display_results and display is not None:
            title = pd.DataFrame({
                "Period": [pname], 
                "Start": [str(pstart_dt.date())], 
                "End": [("today" if pend_dt is None else str(full_period_end.date()))]
            })
            display(title)
            display(_format_styler(df))

    return results


# -------------------------
# Rolling analytics helpers
# -------------------------
def _to_returns(px: pd.Series, freq: str = "D") -> pd.Series:
    """
    Convert price series to returns.
    freq:
      - "D": daily returns
      - "W": weekly returns (Fri close)
      - "M": monthly returns
    """
    px = px.dropna()
    if freq == "D":
        r = px.pct_change()
    elif freq == "W":
        r = px.resample("W-FRI").last().pct_change()
    elif freq == "M":
        r = px.resample("M").last().pct_change()
    else:
        raise ValueError("freq must be one of {'D','W','M'}")
    return r.replace([np.inf, -np.inf], np.nan).dropna()


def rolling_corr(etf_r: pd.Series, bmk_r: pd.Series, window: int) -> pd.Series:
    df = pd.concat([etf_r.rename("etf"), bmk_r.rename("bmk")], axis=1).dropna()
    return df["etf"].rolling(window).corr(df["bmk"])


def rolling_beta_alpha(etf_r: pd.Series, bmk_r: pd.Series, window: int) -> pd.DataFrame:
    """
    OLS in each rolling window:
      etf = alpha + beta*bmk + eps

    Returns DataFrame columns: ['alpha', 'beta', 'r2']
    alpha is per-period (daily/weekly/monthly) alpha, NOT annualized.
    """
    df = pd.concat([etf_r.rename("etf"), bmk_r.rename("bmk")], axis=1).dropna()

    # Rolling means
    mx = df["bmk"].rolling(window).mean()
    my = df["etf"].rolling(window).mean()

    # Rolling cov/var
    cov_xy = df["bmk"].rolling(window).cov(df["etf"])
    var_x  = df["bmk"].rolling(window).var()

    beta = cov_xy / var_x
    alpha = my - beta * mx

    # Rolling R^2: correlation squared is simpler and more stable
    corr = df["etf"].rolling(window).corr(df["bmk"])
    r2 = corr ** 2

    out = pd.DataFrame({"alpha": alpha, "beta": beta, "r2": r2})
    return out


def annualize_alpha(alpha_per_period: pd.Series, periods_per_year: int) -> pd.Series:
    """
    Convert per-period alpha (e.g., daily) into annualized additive approximation.
    For small alpha this is fine. If you prefer compounding, use exp/log.
    """
    return alpha_per_period * periods_per_year


def tracking_error(etf_r: pd.Series, bmk_r: pd.Series, window: int, periods_per_year: int) -> pd.Series:
    """
    Rolling annualized tracking error: std(etf - bmk) * sqrt(periods_per_year)
    """
    df = pd.concat([etf_r.rename("etf"), bmk_r.rename("bmk")], axis=1).dropna()
    active = (df["etf"] - df["bmk"])
    return active.rolling(window).std(ddof=1) * np.sqrt(periods_per_year)


def information_ratio(
    alpha_per_period: pd.Series,
    te_annualized: pd.Series,
    periods_per_year: int
) -> pd.Series:
    """
    Rolling Information Ratio:
      IR = annualized_alpha / annualized_tracking_error
    alpha_per_period should align with te_annualized index.
    """
    ann_alpha = annualize_alpha(alpha_per_period, periods_per_year)
    return ann_alpha / te_annualized.replace(0, np.nan)


def conditional_corr(
    etf_r: pd.Series,
    bmk_r: pd.Series,
    window: int,
    condition: str = "down"
) -> pd.Series:
    """
    Rolling correlation computed only on observations inside each window
    where benchmark return is <0 (down) or >0 (up).

    Note: This is more expensive than plain rolling corr because the set
    of points changes per window. For typical ETF daily data it's fine.
    """
    df = pd.concat([etf_r.rename("etf"), bmk_r.rename("bmk")], axis=1).dropna()

    def _corr_in_window(idx: pd.Index) -> float:
        x = df.loc[idx]
        if condition == "down":
            sub = x[x["bmk"] < 0]
        elif condition == "up":
            sub = x[x["bmk"] > 0]
        else:
            raise ValueError("condition must be 'down' or 'up'")
        if len(sub) < max(10, window // 5):  # require some minimum points
            return np.nan
        return float(sub["etf"].corr(sub["bmk"]))

    # Apply rolling to one column, use its index to get full DataFrame slice
    return df["etf"].rolling(window).apply(lambda w: _corr_in_window(w.index), raw=False)


In [2]:
ticker_list = ["VTI","QQQ","SPMO","AVUV", "VXUS", "AVDV", "SPGP", "QUAL", "USMV", "VONG", "VUG", "VDC", "DYNF", "DUHP", "MTUM"]

In [6]:
periods = {
    "DotCom Bubble Drawdown": [
      "2001-06-02",
      "2004-12-04",
      "2002-10-09"
    ],
    "2005 Bull": [
      "2004-12-04",
      "2006-04-08"
    ],
    "2006 Inflation Consolidation": [
      "2006-04-08",
      "2006-11-26",
      "2006-06-13"
    ],
    "2007 Bull": [
      "2006-11-26",
      "2007-06-13"
    ],
    "GFC Drawdown": [
      "2007-06-13",
      "2012-04-26",
      "2009-03-09"
    ],
    "Eurozone Consolidation": [
      "2012-03-02",
      "2012-10-14",
      "2012-06-04"
    ],
    "2012–2015 Bull": [
      "2012-10-14",
      "2015-05-23"
    ],
    "2015-2016 Selloff Drawdown": [
      "2015-05-23",
      "2016-09-15",
      "2016-02-11"
    ],
    "2016-2017 Bull": [
      "2016-09-15",
      "2017-12-26"
    ],
    "Volmageddon Drawdown": [
      "2017-12-26",
      "2019-05-23",
      "2018-12-24"
    ],
    "2019-2020 Bull": [
      "2019-05-23",
      "2020-01-19"
    ],
    "Covid Drawdown": [
      "2020-01-19",
      "2020-09-28",
      "2020-03-23"
    ],
    "leadership Rotation Consolidation": [
      "2020-08-02",
      "2020-12-27",
      "2020-09-23"
    ],
    "2021 Bull": [
      "2020-12-27",
      "2021-12-03"
    ],
    "2022 Rate Hike Drawdown": [
      "2021-12-03",
      "2024-03-02",
      "2022-10-12"
    ],
    "2024 Bull": [
      "2024-03-02",
      "2024-06-16"
    ],
    "2024 Consolidation": [
      "2024-06-16",
      "2024-12-06",
      "2024-08-05"
    ],
    "2025 Transition Bull": [
      "2024-12-06",
      "2025-01-19"
    ],
    "Trump Tariffs Drawdown": [
      "2025-01-19",
      "2025-08-25",
      "2025-04-08"
    ],
    "AI Bull": [
      "2025-08-25",
      None
    ]
  }

In [9]:
analyze_regimes(ticker_list, periods=periods, benchmark="VTI", rf=0.0)
print("done")

$QUAL: possibly delisted; no price data found  (1d 2001-06-02 -> 2004-12-04) (Yahoo error = "Data doesn't exist for startDate = 991454400, endDate = 1102136400")
$AVDV: possibly delisted; no price data found  (1d 2001-06-02 -> 2004-12-04) (Yahoo error = "Data doesn't exist for startDate = 991454400, endDate = 1102136400")
$AVUV: possibly delisted; no price data found  (1d 2001-06-02 -> 2004-12-04) (Yahoo error = "Data doesn't exist for startDate = 991454400, endDate = 1102136400")
$VXUS: possibly delisted; no price data found  (1d 2001-06-02 -> 2004-12-04) (Yahoo error = "Data doesn't exist for startDate = 991454400, endDate = 1102136400")
$DUHP: possibly delisted; no price data found  (1d 2001-06-02 -> 2004-12-04) (Yahoo error = "Data doesn't exist for startDate = 991454400, endDate = 1102136400")
$VONG: possibly delisted; no price data found  (1d 2001-06-02 -> 2004-12-04) (Yahoo error = "Data doesn't exist for startDate = 991454400, endDate = 1102136400")
$USMV: possibly delisted; no

,Period,Start,End
0,DotCom Bubble Drawdown,2001-06-02,2004-12-04


,DataAvailability,CAGR,AnnVol,Martin,CapitalEfficiency,MaxDD,UlcerIndex,VolatilityDrag,PctTimeNearHighs,TrendPersistence,PctDaysUnderwater,AvgUnderwaterDays,P95UnderwaterDays,MaxUnderwaterDays,TTR_Days,RecoverySlope_Ann,RecoveryEfficiency,RecoveredToPeak_inWindow,ExpectedShortfall,ConditionalCorr_Up,ConditionalCorr_Down,TrackingError_Ann,InformationRatio,UpsideCapture,DownsideCapture,PeriodCorr,PeriodBeta,PeriodAlpha_per_period,PeriodAlpha_Ann,PeriodR2
VTI,99.0%,2.72%,18.96%,0.182,0.078,-34.66%,14.93,0.600,30.42%,47.50%,97.4%,65,355,635,463,26.06%,1.530,1,-2.55%,1.000,1.000,0.00%,—,1.000,1.000,1.000,1.000,0.00%,0.00%,1.000
QQQ,100.0%,-1.70%,32.54%,-0.055,-0.030,-56.11%,30.89,-0.483,1.84%,48.20%,99.1%,288,774,860,—,24.91%,1.785,0,-4.32%,0.754,0.690,19.14%,-0.157,6.556,1.020,0.853,1.463,-0.01%,-3.01%,0.727
VUG,24.3%,3.92%,12.08%,0.790,0.354,-11.08%,4.96,0.841,60.75%,45.00%,94.9%,51,169,199,111,30.55%,1.125,1,-1.72%,0.748,0.625,6.70%,-0.544,0.823,0.935,0.840,0.878,-0.01%,-3.64%,0.706
VDC,24.3%,8.89%,9.83%,2.429,1.090,-8.16%,3.66,0.947,77.10%,53.65%,89.7%,24,92,124,—,70.43%,1.055,0,-1.23%,0.477,0.380,10.36%,0.451,0.364,0.439,0.541,0.460,0.02%,4.68%,0.292


$QUAL: possibly delisted; no price data found  (1d 2004-12-04 -> 2006-04-08) (Yahoo error = "Data doesn't exist for startDate = 1102136400, endDate = 1144468800")
$SPGP: possibly delisted; no price data found  (1d 2004-12-04 -> 2006-04-08) (Yahoo error = "Data doesn't exist for startDate = 1102136400, endDate = 1144468800")
$DUHP: possibly delisted; no price data found  (1d 2004-12-04 -> 2006-04-08) (Yahoo error = "Data doesn't exist for startDate = 1102136400, endDate = 1144468800")
$SPMO: possibly delisted; no price data found  (1d 2004-12-04 -> 2006-04-08) (Yahoo error = "Data doesn't exist for startDate = 1102136400, endDate = 1144468800")
$MTUM: possibly delisted; no price data found  (1d 2004-12-04 -> 2006-04-08) (Yahoo error = "Data doesn't exist for startDate = 1102136400, endDate = 1144468800")
$AVUV: possibly delisted; no price data found  (1d 2004-12-04 -> 2006-04-08) (Yahoo error = "Data doesn't exist for startDate = 1102136400, endDate = 1144468800")
$VONG: possibly delist

,Period,Start,End
0,2005 Bull,2004-12-04,2006-04-08


...

,Period,Start,End
0,Trump Tariffs Drawdown,2025-01-19,2025-08-25


,DataAvailability,CAGR,AnnVol,Martin,CapitalEfficiency,MaxDD,UlcerIndex,VolatilityDrag,PctTimeNearHighs,TrendPersistence,PctDaysUnderwater,AvgUnderwaterDays,P95UnderwaterDays,MaxUnderwaterDays,TTR_Days,RecoverySlope_Ann,RecoveryEfficiency,RecoveredToPeak_inWindow,ExpectedShortfall,ConditionalCorr_Up,ConditionalCorr_Down,TrackingError_Ann,InformationRatio,UpsideCapture,DownsideCapture,PeriodCorr,PeriodBeta,PeriodAlpha_per_period,PeriodAlpha_Ann,PeriodR2
VTI,100.0%,12.41%,23.25%,1.957,0.643,-19.30%,6.34,0.814,65.10%,43.45%,87.2%,16,63,88,80,96.52%,1.239,1,-3.26%,1.000,1.000,0.00%,—,1.000,1.000,1.000,1.000,-0.00%,-0.00%,1.000
QQQ,100.0%,16.42%,27.58%,2.105,0.721,-22.77%,7.80,0.802,64.43%,45.58%,83.2%,12,53,85,77,132.92%,1.295,1,-3.89%,0.977,0.951,6.94%,0.329,1.243,1.096,0.977,1.159,0.01%,2.28%,0.955
SPMO,100.0%,30.69%,27.24%,4.955,1.525,-20.13%,6.19,0.879,67.79%,39.31%,75.8%,6,11,60,39,327.22%,1.252,1,-3.79%,0.955,0.945,7.83%,1.802,1.295,1.038,0.964,1.130,0.06%,14.12%,0.929
AVUV,100.0%,-1.42%,27.01%,-0.127,-0.056,-25.08%,11.16,-0.642,26.17%,46.26%,99.3%,148,148,148,—,68.13%,1.291,0,-3.80%,0.850,0.814,12.52%,-0.998,1.053,1.137,0.887,1.030,-0.05%,-12.50%,0.786
VXUS,100.0%,38.21%,18.12%,14.657,2.814,-13.58%,2.61,0.952,93.96%,48.98%,69.8%,5,16,30,24,362.97%,1.157,1,-2.36%,0.810,0.694,13.23%,1.861,0.688,0.606,0.823,0.642,0.10%,24.62%,0.678
AVDV,100.0%,62.53%,19.55%,24.285,4.411,-14.17%,2.57,0.962,95.30%,49.65%,66.4%,4,11,26,21,526.05%,1.165,1,-2.44%,0.713,0.671,15.70%,2.627,0.668,0.430,0.744,0.625,0.16%,41.25%,0.553
SPGP,100.0%,4.65%,25.85%,0.570,0.209,-22.29%,8.16,0.578,44.97%,44.90%,94.6%,24,95,123,106,82.14%,1.287,1,-3.71%,0.933,0.897,8.53%,-0.842,1.041,1.081,0.945,1.051,-0.03%,-7.19%,0.893
QUAL,100.0%,7.72%,21.04%,1.299,0.430,-17.96%,5.94,0.772,65.10%,48.30%,92.6%,20,77,105,106,60.10%,1.219,1,-2.90%,0.979,0.966,4.62%,-0.679,0.880,0.960,0.983,0.890,-0.01%,-3.14%,0.967
USMV,100.0%,8.66%,15.17%,3.952,0.926,-9.36%,2.19,0.878,96.64%,35.37%,90.6%,10,42,54,41,82.94%,1.103,1,-1.94%,0.745,0.644,14.95%,0.146,0.443,0.590,0.775,0.506,0.01%,2.18%,0.601
VONG,100.0%,15.78%,27.60%,1.957,0.684,-23.05%,8.06,0.796,63.09%,38.46%,85.9%,12,56,105,79,130.69%,1.300,1,-3.83%,0.968,0.949,7.31%,0.245,1.271,1.111,0.973,1.155,0.01%,1.79%,0.947


,Period,Start,End
0,AI Bull,2025-08-25,today


,DataAvailability,CAGR,AnnVol,Martin,CapitalEfficiency,MaxDD,UlcerIndex,VolatilityDrag,PctTimeNearHighs,TrendPersistence,PctDaysUnderwater,AvgUnderwaterDays,P95UnderwaterDays,MaxUnderwaterDays,TTR_Days,RecoverySlope_Ann,RecoveryEfficiency,RecoveredToPeak_inWindow,ExpectedShortfall,ConditionalCorr_Up,ConditionalCorr_Down,TrackingError_Ann,InformationRatio,UpsideCapture,DownsideCapture,PeriodCorr,PeriodBeta,PeriodAlpha_per_period,PeriodAlpha_Ann,PeriodR2
VTI,100.0%,25.24%,11.68%,17.521,4.778,-5.28%,1.44,0.970,98.85%,56.47%,72.4%,6,21,29,20,98.13%,1.056,1,-1.68%,1.000,1.000,0.00%,—,1.000,1.000,1.000,1.000,0.00%,0.00%,1.000
QQQ,100.0%,31.50%,16.04%,12.466,3.998,-7.88%,2.53,0.955,93.10%,51.76%,74.7%,8,28,40,—,57.07%,0.846,0,-2.36%,0.810,0.931,6.48%,-0.183,1.270,1.204,0.939,1.289,-0.00%,-1.19%,0.881
SPMO,100.0%,15.07%,15.00%,5.005,1.895,-7.96%,3.01,0.926,87.36%,49.41%,83.9%,15,37,40,—,56.87%,0.811,0,-2.20%,0.732,0.890,6.58%,-1.779,1.055,1.179,0.908,1.166,-0.05%,-11.71%,0.824
AVUV,100.0%,19.09%,17.51%,6.989,2.512,-7.60%,2.73,0.919,94.25%,48.19%,88.5%,13,42,52,13,362.85%,1.082,1,-2.24%,0.573,0.682,11.91%,-0.545,0.972,1.044,0.736,1.103,-0.03%,-6.49%,0.542
VXUS,100.0%,26.84%,10.61%,20.148,5.606,-4.79%,1.33,0.977,100.00%,56.47%,70.1%,5,14,18,20,85.55%,1.050,1,-1.46%,0.612,0.737,6.33%,1.027,0.825,0.784,0.843,0.766,0.03%,6.49%,0.711
AVDV,100.0%,40.15%,11.64%,32.746,10.706,-3.75%,1.23,0.980,100.00%,47.06%,70.1%,5,12,13,6,398.05%,1.039,1,-1.54%,0.522,0.509,8.69%,2.016,0.887,0.702,0.722,0.720,0.07%,17.53%,0.522
SPGP,100.0%,10.40%,13.33%,4.431,1.373,-7.58%,2.35,0.917,94.25%,49.41%,86.2%,12,38,46,21,157.39%,1.082,1,-1.86%,0.631,0.829,6.91%,-1.691,0.847,1.018,0.855,0.976,-0.05%,-11.69%,0.731
QUAL,100.0%,23.23%,10.44%,15.916,4.712,-4.93%,1.46,0.974,100.00%,57.65%,74.7%,8,21,26,14,148.45%,1.052,1,-1.37%,0.810,0.916,3.90%,0.474,0.872,0.882,0.944,0.844,0.01%,1.85%,0.891
USMV,100.0%,7.12%,7.91%,5.510,2.198,-3.24%,1.29,0.956,100.00%,47.06%,87.4%,11,40,53,48,18.86%,1.033,1,-0.96%,0.353,0.245,10.16%,-0.092,0.355,0.443,0.518,0.351,-0.00%,-0.94%,0.268
VONG,100.0%,26.98%,15.11%,9.943,3.604,-7.49%,2.71,0.954,93.10%,50.59%,79.3%,12,33,40,—,47.90%,0.768,0,-2.17%,0.787,0.923,5.84%,-0.520,1.202,1.187,0.937,1.212,-0.01%,-3.04%,0.877


done
